In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

ds = load_dataset("dair-ai/emotion", "split")

id2label = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=num_labels)
ß
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True, batch_size=16)

/Users/yara/Documents/Projects/EmotionDetection/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yara/Documents/Projects/EmotionDetection/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.wei

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) 

    f1 = f1_score(labels, predictions, average="micro")
    accuracy = accuracy_score(labels, predictions)

    return {"f1": f1, "accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    metric_for_best_model="f1", 
    load_best_model_at_end=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics, 
)

In [ ]:
trainer.evaluate()

{'eval_loss': 2.075010061264038,
 'eval_model_preparation_time': 0.0234,
 'eval_f1': 0.275,
 'eval_accuracy': 0.275,
 'eval_runtime': 64.8063,
 'eval_samples_per_second': 30.861,
 'eval_steps_per_second': 1.929}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,F1,Accuracy
1,0.234100,0.196688,0.023400,0.936000,0.936000
2,0.136700,0.135690,0.023400,0.945500,0.945500


TrainOutput(global_step=2000, training_loss=0.30065270614624023, metrics={'train_runtime': 3111.1194, 'train_samples_per_second': 10.286, 'train_steps_per_second': 0.643, 'total_flos': 8419856154624000.0, 'train_loss': 0.30065270614624023, 'epoch': 2.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/BertEmotionDetection")

In [ ]:
trainer.evaluate()